# Objectives
   TheMovieDB.org provide enough information about the movie and actor, but not detail information about the charter.
   We well use wikipedia.org to get more detail information about the character role and a second source for the movie plot.


In [35]:
import os
import sys
import time
from datetime import datetime
import urllib.parse
import http.client
import json
import logging
logger = logging.getLogger('actorInfo')
logger.setLevel(logging.INFO)

# Use the wikipedia library to extract the wikipedia.org site

In [36]:
import wikipedia
#print (wikipedia.summary("Wikipedia"))
# Wikipedia (/ˌwɪkɨˈpiːdiə/ or /ˌwɪkiˈpiːdiə/ WIK-i-PEE-dee-ə) is a collaboratively edited, multilingual, free Internet encyclopedia supported by the non-profit Wikimedia Foundation...
source='wikipedia'

In [37]:
path_to_json = './data/cast/themoviedb/'
json_files = [path_to_json+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
len(json_files)
df = pd.concat([pd.read_json(f , dtype=str) for f in json_files], ignore_index = True)


In [38]:
movieDF = df[['movieID','movieTitle']]
movieDF.drop_duplicates(keep = False, inplace = True) 
movieDF.head()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,movieID,movieTitle
0,20829,Funny People
2,38365,Grown Ups
5,76492,Hotel Transylvania
6,87428,That's My Boy
8,232672,Blended


# Extract Movie Info from Wikipedia based on the Top 100 Actors

In [39]:
def searchTitle(movieTitle):
    search_results = wikipedia.search(movieTitle)
    for page in search_results:
        if page == movieTitle:
            #print( 'Matched Page : ', page )
            return page


In [ ]:
output=[]
for index, row in df.iterrows():
    movieTitle= row['movieTitle']
    _movieID = row['movieID']
    print(movieTitle)
    
    try:
        content ={'_movieID':_movieID, 'source': source, 'created_utc_dt': datetime.utcnow().strftime("%m/%d/%Y, %H:%M:%S")} 
        wikiPage = searchTitle(movieTitle)
        if wikiPage:
            wikiContent= wikipedia.page(wikiPage)
            if wikiContent:
                
                summary = wikiContent.summary
                if summary:
                    content.update({'summary': summary})
                    
                plot = wikiContent.section('Plot')
                if plot:
                    content.update({'plot': plot})
                    
                casts = wikiContent.section('Cast')
                if casts:
                    content.update({'cast': casts})
                
        f = open('./data/movies/'+source+'/'+source+'_m_'+_movieID+'.json', 'w+')
        f.write(json.dumps(content))
        f.close()

                  
    except Exception as e:
        logger.error('Failed to retrieve data : '+movieTitle+' :: '+ str(e))


In [ ]:
    wikiContent= wikipedia.page(wikiPage)
    casts = wikipage.section('Cast')
    #print(cast)
    role, desc = cast.split(':')
    actor, character = role.split(' as ')
    if actor == match_actor:
        print(actor, ' played ' , character, ' as ', desc, ' does it match :' , i_character)
        

# Return Actor Biographica Info from Wikipedia

In [105]:
topActors = pd.read_csv('./top100Actors.txt', 
                     sep='\t',
                     header=0, 
                    dtype=str)

topActors.head(15)


,Rank,First Name,Last Name,Domestic Box Office,Movies,Average
0,1,Robert,Downey Jr.,"$5,374,149,735",43,"$124,980,226"
1,2,Samuel L.,Jackson,"$5,234,777,328",60,"$87,246,289"
2,3,Scarlett,Johansson,"$4,859,403,632",28,"$173,550,130"
3,4,Tom,Hanks,"$4,692,293,839",51,"$92,005,762"
4,5,Bradley,Cooper,"$4,263,310,067",26,"$163,973,464"
5,6,Tom,Cruise,"$3,937,740,350",42,"$93,755,723"
6,7,Zoe,Saldana,"$3,932,967,185",26,"$151,267,969"
7,8,Chris,Hemsworth,"$3,915,069,280",22,"$177,957,695"
8,9,Chris,Pratt,"$3,866,023,741",17,"$227,413,161"
9,10,Harrison,Ford,"$3,850,910,074",40,"$96,272,752"


In [106]:
actorsDF = topActors[['Rank','First Name','Last Name']]
actorsDF['Full Name']= actorsDF['First Name']+' '+actorsDF['Last Name']
actorsDF = actorsDF.drop(['First Name','Last Name'], axis=1)
actors = actorsDF.values.tolist()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [111]:
for actor in actors:
    try:
        _actorID= actor[0]
        actorFullName = actor[1]
        logger.info('Retrieving Wikipedia bio data for : '+actorFullName)
        
        wikiPage = searchTitle(actorFullName)
        wikiContent= wikipedia.page(wikiPage)
        wikiText = wikiContent.content
        sections = wikiText.split('\n\n\n== ')
        bio ={}
        bio.update( {'_id': _actorID, 'name':actorFullName,  'source': source, 'created_utc_dt': datetime.utcnow().strftime("%m/%d/%Y, %H:%M:%S")})
        for section in sections:
            
            if section.find('==' )>0:
                #print('Section Head  :  ' , section)
                subsections = section.split('==',1)
                bio.update( {subsections[0].strip(): subsections[1].strip()})
            else:
                bio.update({'biography': section})
        
        if not os.path.exists('./data/person/'+source):
            os.makedirs('./data/person/'+source)
            
        f = open('./data/person/'+source+'/'+source+'_a_'+_actorID+'.json', 'w+')
        f.write(json.dumps(bio))
        f.close()

                  
    except Exception as e:
        logger.error('Failed to retrieve data : '+actorFullName+' :: '+ str(e))
        

Failed to retrieve data : Chris Evans :: Either a title or a pageid must be specified
